Find the markdown blocks that say interaction required! The notebook should take care of the rest!

# Import libs

In [1]:
import sys
import os
sys.path.append('..')
from eflow.foundation import DataPipeline,DataFrameTypes
from eflow.model_analysis import ClassificationAnalysis
from eflow.utils.modeling_utils import optimize_model_grid
from eflow.utils.eflow_utils import get_type_holder_from_pipeline, remove_unconnected_pipeline_segments
from eflow.utils.pandas_utils import data_types_table
from eflow.auto_modeler import AutoCluster

import pandas as pd
import numpy as np
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import copy
import pickle
from IPython.display import clear_output

ModuleNotFoundError: No module named 'pyclustering'

In [ ]:
# # Additional add ons
# !pip install pandasgui
# !pip install pivottablejs
# clear_output()

In [ ]:
%matplotlib notebook
%matplotlib inline

## Declare Project Variables

### Interaction required

In [ ]:
dataset_path = "Datasets/titanic_train.csv"

# -----
dataset_name = "Titanic Data"
pipeline_name = "Titanic Pipeline"

# -----


# -----
notebook_mode = True

## Clean out segment space

In [ ]:
remove_unconnected_pipeline_segments()

# Import dataset

In [ ]:
df = pd.read_csv(dataset_path)
shape_df = pd.DataFrame.from_dict({'Rows': [df.shape[0]],
                                   'Columns': [df.shape[1]]})
display(shape_df)
display(df.head(30))

In [ ]:
data_types_table(df)

# Loading and init df_features

In [ ]:
# Option: 1
# df_features = get_type_holder_from_pipeline(pipeline_name)

In [ ]:
# Option: 2
df_features = DataFrameTypes()
df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")

In [ ]:
df_features.display_features(display_dataframes=True,
                             notebook_mode=notebook_mode)

# Any extra processing before eflow DataPipeline

# Setup pipeline structure

### Interaction Required

In [ ]:
main_pipe = DataPipeline(pipeline_name,
                         df,
                         df_features)

In [ ]:
main_pipe.perform_pipeline(df,
                           df_features)

In [ ]:
df

# Generate clustering models with automodeler

In [ ]:
auto_cluster = AutoCluster(df,
                           df_features,
                           project_sub_dir=dataset_name,
                           overwrite_full_path=None,
                           notebook_mode=True)

### Temporialy remove dataframe to save RAM for processing

In [ ]:
del df

# Inspect Hierarchical models

In [ ]:
# auto_cluster.visualize_hierarchical_clustering()

In [ ]:
auto_cluster.create_kmeans_models_with_elbow_graph()

In [ ]:
raise ValueError("Testing")

In [ ]:
df = pd.read_csv(dataset_path)
df

In [ ]:
df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")
df_features.display_features(display_dataframes=True,
                             notebook_mode=notebook_mode)

In [ ]:
main_pipe.perform_pipeline(df,
                           df_features)

In [ ]:
df

In [ ]:
from kneed import DataGenerator, KneeLocator

x = [5346.0, 4464.47220442314, 3650.8164825776444, 2883.4865344266404, 2353.5226410046207, 1923.7811636297681, 1668.8974296738902, 1472.2444437394222, 1280.4754069907601, 1149.2682037351174, 1057.4340058596358, 959.3105185259967, 881.3187084234528, 806.0797704821307]
kneedle = KneeLocator(x, [i for i in range(0,len(x))], curve='concave', direction='increasing', interp_method='polynomial', online=True)

# average knee point
print(kneedle.knee)
round(sum(knees) / len(knees), 3)
63.583

In [ ]:
inertias = [5346.0, 4464.47220442314, 3650.8164825776444]
inertias_matrix = np.matrix(inertias)

In [ ]:
inertias_matrix

In [ ]:
inertias_matrix = np.vstack([inertias_matrix, [5000, 4000, 3680]])
inertias_matrix = np.vstack([inertias_matrix, [7000, 6000, 4680]])
inertias_matrix = np.vstack([inertias_matrix, [5700, 5300, 4680]])

inertias_matrix

In [ ]:
average_elbow_inertias = inertias_matrix.mean(0)
average_elbow_inertias

In [ ]:
knee_vote = []

for vector in inertias_matrix:
    knee_vote.append(np.absolute(vector - average_elbow_inertias).sum())

best_elbow_index = np.array(knee_vote).argmin()
best_of_elbows

In [ ]:
knee_vote

In [ ]:
def create_kmeans_models_with_elbow_graph(self,
                                          random_state=None):
    """
        Generate models based on the found 'elbow' of the interia values.
    """

    elbow_inertias_matrix = None
    inertias_matrix = None
    elbow_models = []
    plt.figure(figsize=(13, 6))
    plt.title("All possible Kmeans Elbow's", fontsize=15)
    plt.xlabel('Number of clusters, k')
    plt.ylabel('Inertia')
    ks = range(1, 15)
    plt.xticks(ks)

    for i in range(0,200):

        inertias = []
        k_models = []

        for k in ks:
            # Create a KMeans instance with k clusters: model
            model = KMeans(n_clusters=k,
                           random_state=random_state).fit(self.__scaled)

            # Append the inertia to the list of inertias
            inertias.append(model.inertia_)
            k_models.append(model)

        # Plot ks vs inertias
        plt.plot(ks,
                 inertias,
                 '-o',
                 color='#367588')

        elbow_cluster = KneeLocator([i for i in range(1,
                                          len(inertias) + 1)],
                                  inertias,
                                  curve='convex',
                                  direction='decreasing').knee

        for k_val in [elbow_cluster - 1, elbow_cluster, elbow_cluster + 1]:
            plt.plot(ks[k_val - 1], inertias[k_val - 1], 'r*')

        elbow_models.append(k_models[elbow_cluster - 2:elbow_cluster + 1])

        if isinstance(elbow_inertias_matrix,type(None)):
            inertias_matrix = np.matrix(inertias)
            inertias = inertias[k_val - 2: k_val + 1]

            elbow_inertias_matrix = np.matrix(inertias)

        else:
            inertias_matrix = np.vstack([inertias_matrix, inertias])
            inertias = inertias[k_val - 2: k_val + 1]

            elbow_inertias_matrix = np.vstack([elbow_inertias_matrix, inertias])

        del inertias
        del k_models
        del elbow_cluster

    plt.show()
    plt.close("all")

    plt.figure(figsize=(13, 6))
    plt.title("Best of all Kmeans Elbow", fontsize=15)
    plt.xlabel('Number of clusters, k')
    plt.ylabel('Inertia')
    plt.xticks(ks)

    average_elbow_inertias = elbow_inertias_matrix.mean(0)

    knee_vote = []
    for vector in elbow_inertias_matrix:
        knee_vote.append(
            np.absolute(vector - average_elbow_inertias).sum())

    best_elbow_index = np.array(knee_vote).argmin()

    plt.plot(ks,
             inertias_matrix[best_elbow_index].tolist()[0],
             '-o',
             color='#367588')


    for model in elbow_models[best_elbow_index]:
        k_val = model.n_clusters
        self.__all_cluster_models["Kmeans_Cluster_" + str(k_val)] = model
        plt.plot(ks[k_val - 1], inertias_matrix[best_elbow_index].tolist()[0][k_val - 1], 'r*')

        # print("Successfully generate Kmeans model on k_val={0}".format(k_val))

    plt.show()
    plt.close()